#  Routing with IPv4
<i>Adapted for use with FABRIC from [IPv4 Routing Assignment](https://groups.geni.net/geni/wiki/GENIEducation/SampleAssignments/IPRouting/Procedure)</i>
        
<b> Prerequisites  </b>
    
* You need to have your FABRIC bastion host key pair set up to do this tutorial. If you have not already set this up, follow steps 1-5 at https://github.com/fabric-testbed/teaching-materials/blob/main/Getting%20Started.md#section-1-get-started.
* You are comfortable using ssh and executing basic commands using a UNIX shell. [Tips about how to login to hosts.](https://github.com/fabric-testbed/teaching-materials/blob/main/Getting%20Started.md)

This is the second step in this assignment, to go to the previous step go to slice creation notebook or click [Here](./CreateSlice.ipynb) 
<br><img src="./figures/AA.png">
<br>The goal of this exercise is to setup the routing as indicated in the figure above; i.e. packets from A sent to IP address 192.168.2.12 on node C should be routed via node B. In order to create this routing behavior you will need to modify the routing tables in your nodes using the [linux route command](https://www.hscripts.com/tutorials/linux-commands/route.html)

## 1. Set up the Experiment

### 1.1 Reserve Resources

#### Import the Fabric API

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

import json
import traceback

####  Retrieve Slice
Create the slice at the [Create Slice Notebook](./CreateSlice.ipynb) and import it here.


In [ ]:
slice_name = 'IPv4_Routing'
try:
    slice = fablib.get_slice(slice_name)   
    slice.list_nodes()
except Exception as e:
    print(f"Get Slices Fail: {e}")

## 1.2 Setup the node network
### 1.2.1 Give each node an address

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

subnet1 = IPv4Network("192.168.1.0/24")
subnet2 = IPv4Network("192.168.2.0/24")
subnet3 = IPv4Network("192.168.3.0/24")

Ctrl_Network = 'Controll_Conn'

In [ ]:
# configure nodes
try:
    NDA = slice.get_node('ND_A')
    NDB = slice.get_node('ND_B')
    NDC = slice.get_node('ND_C')
    
    #NDA
    A_iface = NDA.get_interface(network_name='Lan_AB')  
    A_iface.ip_addr_add(addr='192.168.1.10', subnet=subnet1)
    A_iface = NDA.get_interface(network_name='Lan_CA')  
    A_iface.ip_addr_add(addr='192.168.3.10', subnet=subnet3)
    
    #NDB
    B_iface = NDB.get_interface(network_name='Lan_AB')  
    B_iface.ip_addr_add(addr='192.168.1.11', subnet=subnet1)
    B_iface = NDB.get_interface(network_name='Lan_BC')  
    B_iface.ip_addr_add(addr='192.168.2.11', subnet=subnet2)
    
    #NDC
    C_iface = NDC.get_interface(network_name='Lan_CA')  
    C_iface.ip_addr_add(addr='192.168.3.12', subnet=subnet3)    
    C_iface = NDC.get_interface(network_name='Lan_BC')  
    C_iface.ip_addr_add(addr='192.168.2.12', subnet=subnet2)    
    
    
    for node in slice.get_nodes():  
        for interface in node.get_interfaces():
            stdout, stderr = node.execute(f'sudo ip link set dev { interface.get_device_name()} up')
        
        print(f"Network status at node {node.get_name()}")
        node.execute("ip a")      
        
        #install route and traceroute
        node.execute("sudo apt update")
        node.execute("sudo apt install -y net-tools")
        node.execute("sudo apt install -y traceroute")
        
        node.execute("sudo sysctl -w net.ipv4.ip_forward=1")
    
except Exception as e:
    print(f"Fail: {e}")

## 2. Run Assignment
### 2.1 Routing Tables
The goal of this assignment is to understand what the effects of routing are and understand what the table represents. In this assignment you will set up the routing in the nodes as it was indicated in the figure above. Packets from A sent to IP address 192.168.2.12 on node C should be routed via node B. In order to create this routing behavior you will need to modify the routing tables in your nodes using the [linux route command](https://www.hscripts.com/tutorials/linux-commands/route.html)
1. What happens when you traceroute from A to IP address 192.168.2.12 before you setup the static routes? Why?
	(Include the output of the traceroute in your writeup)
2. Setup the routing from A to 192.68.2.12 so that it goes through B. Was it enough to just modify the routing tables? What else did you need to change in order for the traffic to flow? Ensure that you have connectivity by running a ping from A to 192.168.2.12
	(In your writeup include all the commands you ran and a screenshot of the routing configuration)
3. What happens when you traceroute from A to IP address 192.168.2.12 after you setup the static routes?
	(In your writeup include a screenshot of the traceroute output.)



## Cleanup Resources

In [ ]:
try:
    #To delete the slice change "CHECK" to "True", this is to prevent accidental slice deletion
    CHECK = False
    if (CHECK):
        slice = fablib.get_slice(slice_name)
        slice.delete()
    else:
        print("Change the variable to delete slice")
except Exception as e:
    print(f"Fail: {e}")